In [1]:
'''
alternative least square

'''

'\nalternative least square\n\n'

In [2]:
import findspark
findspark.init() # spark_home目录
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark06").master("local[2]"). \
config("spark.sql.warehouse.dir","file:///E:/input/spark/warehouse").getOrCreate()

#data = spark.read.csv("file:///e:/Download/mllib/seeds_dataset.txt",inferSchema=True,header=True)

In [3]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
data = spark.read.csv("file:///e:/Download/mllib/ratings.csv",inferSchema=True, header=True) # 文本文件
data.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [5]:
data.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      6|   2.0|980730861|
|     1|     22|   3.0|980731380|
|     1|     32|   2.0|980731926|
|     1|     50|   5.0|980732037|
|     1|    110|   4.0|980730408|
|     1|    164|   3.0|980731766|
|     1|    198|   3.0|980731282|
|     1|    260|   5.0|980730769|
|     1|    296|   4.0|980731208|
|     1|    303|   3.0|980732235|
|     1|    318|   3.0|980731417|
|     1|    350|   3.0|980731745|
|     1|    366|   2.0|980731621|
|     1|    367|   4.0|980731380|
|     1|    431|   2.0|980731312|
|     1|    432|   2.0|980732235|
|     1|    451|   1.0|980731789|
|     1|    457|   4.0|980730816|
|     1|    474|   3.0|980730816|
|     1|    480|   4.0|980731903|
+------+-------+------+---------+
only showing top 20 rows



In [6]:
data.describe().show()

+-------+-----------------+------------------+------------------+--------------------+
|summary|           userId|           movieId|            rating|           timestamp|
+-------+-----------------+------------------+------------------+--------------------+
|  count|           100023|            100023|            100023|              100023|
|   mean|341.7607650240445|  8613.12344160843| 3.491361986743049|1.0915203641766994E9|
| stddev|193.8497546127998|19736.006106155033|1.0679416920809688| 1.633576771608459E8|
|    min|                1|                 1|               0.5|           828504918|
|    max|              706|            129651|               5.0|          1427754939|
+-------+-----------------+------------------+------------------+--------------------+



In [7]:
train,test = data.randomSplit([0.8,0.2])

In [8]:
als = ALS(rank=10,
    maxIter=10,
    regParam=0.1,
    numUserBlocks=10,
    numItemBlocks=10,
    implicitPrefs=False,
    alpha=1.0,
    userCol='user',
    itemCol='item',
    seed=None,
    ratingCol='rating',
    nonnegative=False,
    checkpointInterval=10,
    intermediateStorageLevel='MEMORY_AND_DISK',
    finalStorageLevel='MEMORY_AND_DISK',
    coldStartStrategy='nan',) # 将评分矩阵分解为两个Low rank举证

In [9]:
als = ALS(maxIter=10,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [10]:
model = als.fit(train)

In [11]:
print(data.select("userId").distinct().count(), model.userFactors.count())

706 706


In [12]:
print(data.select("movieId").distinct().count(),model.itemFactors.count())

8552 7930


In [13]:
preds = model.transform(test) # 预测评分，产生新栏位prediction

In [14]:
preds.show(3)

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|   159|    471|   4.0|1076529703| 4.0452576|
|   182|    471|   3.0| 835697021|   4.90126|
|   484|    471|   2.0| 938783830| 3.4903452|
+------+-------+------+----------+----------+
only showing top 3 rows



In [19]:
preds.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)
 |-- prediction: float (nullable = false)



In [15]:
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction' )

In [16]:
rmse = evaluator.evaluate(preds) # 预测评估

In [22]:
print(rmse) # 不能产生数据

nan


In [44]:
evaluator.evaluate(preds)

nan

In [52]:
rdd = preds.rdd

In [55]:
#rdd.foreach(print)

In [ ]:
preds.select('rating').

In [33]:
single_user = test.filter(test['userId']==11) # 挑选一个测试用户

In [35]:
single_user.show() # 用户实际评分

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|    11|    104|   3.0|1111487971|
|    11|    553|   3.0|1111487980|
|    11|   1610|   3.0|1111487910|
|    11|   2959|   4.0|1111487906|
|    11|   2987|   3.0|1111487939|
|    11|   3793|   3.5|1111488017|
+------+-------+------+----------+



In [36]:
recomendations = model.transform(single_user) # 测试用户推荐

In [39]:
recomendations.orderBy('prediction',ascending=False).show()

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|    11|   2959|   4.0|1111487906| 6.1474605|
|    11|    553|   3.0|1111487980| 5.1136513|
|    11|   1610|   3.0|1111487910|  3.964507|
|    11|   3793|   3.5|1111488017|  3.650897|
|    11|   2987|   3.0|1111487939| 3.0012856|
|    11|    104|   3.0|1111487971|  2.096114|
+------+-------+------+----------+----------+

